In [ ]:
def build_resunet(input_shape=(128, 128, 3)):
    inputs = layers.Input(input_shape)

    def residual_block(x, filters):
        shortcut = x

        x = layers.Conv2D(filters, 3, padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)

        x = layers.Conv2D(filters, 3, padding='same')(x)
        x = layers.BatchNormalization()(x)

        if shortcut.shape[-1] != filters:
            shortcut = layers.Conv2D(filters, 1, padding='same')(shortcut)
            shortcut = layers.BatchNormalization()(shortcut)

        x = layers.add([x, shortcut])
        x = layers.Activation('relu')(x)

        return x

    # Initial convolution
    x = layers.Conv2D(64, 3, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    # Encoder
    x1 = residual_block(x, 64)
    p1 = layers.MaxPooling2D((2, 2))(x1)

    x2 = residual_block(p1, 128)
    p2 = layers.MaxPooling2D((2, 2))(x2)

    x3 = residual_block(p2, 256)
    p3 = layers.MaxPooling2D((2, 2))(x3)

    # Bridge
    x4 = residual_block(p3, 512)

    # Decoder
    u5 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(x4)
    u5 = layers.concatenate([u5, x3])
    x5 = residual_block(u5, 256)

    u6 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(x5)
    u6 = layers.concatenate([u6, x2])
    x6 = residual_block(u6, 128)

    u7 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(x6)
    u7 = layers.concatenate([u7, x1])
    x7 = residual_block(u7, 64)

    outputs = layers.Conv2D(1, 1, activation='sigmoid')(x7)

    model = Model(inputs=inputs, outputs=outputs)
    model.summary()
    return model
